In [1]:
# Python 샘플 코드 #
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# 해당 코드에서 크롤링을 시도할 연도(yyyy00)
this_year = 201500

In [3]:
'''
국토교통부_아파트매매 실거래 상세 자료
활용기간 2023.04.12 까지
LAWD_CD = 11110  # 47290(경산) 지역코드 - 각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리
DEAL_YMD = 200601  # 계약월 - 실거래 자료의 계약년월 6자리
numOfRows = 500  # 한 페이지 결과 수
pageNo = 1  # 페이지번호
'''
def get_apt_data(LAWD_CD, DEAL_YMD, serviceKey, numOfRows=10000):
    # 호출하려는 OpenAPI URL 정의
    data_url = "".join(
        ["http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?",
         "serviceKey=", serviceKey, "&LAWD_CD=", str(LAWD_CD), "&DEAL_YMD=", str(DEAL_YMD), "&numOfRows=",
         str(numOfRows)])

    # 정의된 OpenAPI URL을 호출
    data_response = requests.get(data_url)
    data_xml = BeautifulSoup(data_response.text, "xml")
    data_xml_row = data_xml.find_all("item")
    data_xml_row

    # XML에 포함된 항목들을 리스트로 저장
    list_items = ["거래금액", "건축년도", "년", "도로명", "도로명건물본번호코드", "도로명건물부번호코드",
                  "도로명시군구코드", "도로명일련번호코드", "도로명지상지하코드", "도로명코드",
                  "법정동", "법정동본번코드", "법정동부번코드", "법정동시군구코드", "법정동읍면동코드", "법정동지번코드",
                  "아파트", "월", "일", "일련번호", "전용면적", "지번", "지역코드", "층", "해제사유발생일", "해제여부"]

    # 리스트의 리스트 형태로 데이터프레임에 저장할 자료 만들기
    list_dataframe = []

    for n_obs in range(0, len(data_xml_row)):
        list_obs_values = []  # 동일한 관측치의 데이터는 모두 이 리스트에 저장된다
        for i in range(0, len(list_items)):
            # 특정 항목이 XML에 아예 존재하지 않는 경우가 있으므로 try 구문 사용
            try:
                list_obs_values.append(data_xml_row[n_obs].find(list_items[i]).text)
            except Exception as e:
                list_obs_values.append("")
                print("Error in " + str(n_obs) + "th obs, item: " + list_items[i] + ", " + str(e))
        list_dataframe.append(list_obs_values)  # 리스트의 리스트로 관측치 저장

    # 데이터프레임 형태로 변환
    df_apt = pd.DataFrame(list_dataframe, columns=list_items)
    return df_apt

In [4]:
# 함수 작동 여부 확인
serviceKey_YKim = "%2F2a7hudNIqdsS1Uq7SaicUQYvpno8zZ4M53iUr4PgZnXIDq4SMhbDLNHrk6PfFXNdnb7LuNpEffhq8YXRq2dbQ%3D%3D"
df_tmp1 = get_apt_data(LAWD_CD=11110, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'


In [5]:
# Import TXT file 법정동코드 전체자료 불러오기
#df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
#paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd["LAWD_CD"] = df_lawd_cd["법정동코드"].divmod(100000)[0]
df_lawd_cd_nodup = df_lawd_cd.drop_duplicates(subset=['LAWD_CD']).copy()  # 중복값 제외
df_lawd_cd_nodup_exist = df_lawd_cd_nodup[df_lawd_cd_nodup["폐지여부"] == "존재"].copy()

In [6]:
total_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"]
tmp_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"][3:10]

In [7]:
# 국토교통부_아파트매매 실거래 상세 자료는 2008년 1월부터 자료가 존재한다.
# 그런데 2006년으로 입력해도 자료가 있다.

In [8]:

# 시작 시점과 종료 시점 입력
yyyymm_start = this_year + 1  # 시작 년월
yyyymm_end = this_year + 3  # 종료 년월

list_yyyymm = []
for n_yyyymm in range(yyyymm_start, yyyymm_end+1):
    if (divmod(n_yyyymm, 100)[1] != 0) & (divmod(n_yyyymm, 100)[1] < 13):
        list_yyyymm.append(n_yyyymm)

list_yyyymm.reverse()


# StopWatch: 코드 시작
time_this_code_start = datetime.now()
print("This code started at: " + str(time_this_code_start))

# DataFrame 구성을 위해 공백 자료를 하나 만든다.
df_dataset_null = get_apt_data(LAWD_CD=11120, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

# 전국의 한달 자료 받는데 약 9분 소요
for yyyymm in list_yyyymm:
    df_dataset = df_dataset_null.copy()
    for lawd_cd in total_lawd_cd:
        df_apt = get_apt_data(LAWD_CD=lawd_cd, DEAL_YMD=yyyymm, serviceKey=serviceKey_YKim)
        # df_apt.to_pickle("./data_raw/df_apt_" + str(yyyymm) + "_" + str(lawd_cd) + ".pkl")
        df_dataset = pd.concat([df_dataset, df_apt])
    df_dataset.to_pickle("C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_" + str(yyyymm) + ".pkl")

# StopWatch: 코드 종료
time_this_code_end = datetime.now()
print("This code finished at: " + str(time_this_code_end))
print("Elapsed (in this code): " + str(time_this_code_end - time_this_code_start))

This code started at: 2021-05-13 14:57:31.918402
Error in 83th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 128th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 131th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 134th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 142th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 144th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 149th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 154th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 161th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 162th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 169th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 170th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Erro

Error in 13th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 23th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 28th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 29th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 33th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 41th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 51th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 62th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 83th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 111th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 129th obs, item: 도로명지상지하코드, 'NoneType' object has no a

Error in 279th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 283th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 287th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 288th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 290th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 294th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 295th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 297th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 299th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 301th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 306th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 308th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 312th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 729th obs, item: 법정동본번코드, 'NoneType' object has no attribute 'text'
Error in 729th obs, item: 법정동부번코드, 'NoneType' object has no attribute 'text'
Error in 729th obs, item: 법정동지번코드, 'NoneType' object has no attribute 'text'
Error in 729th obs, item: 지번, 'NoneType' object has no attribute 'text'
Error in 730th obs, item: 법정동본번코드, 'NoneType' object has no attribute 'text'
Error in 730th obs, item: 법정동부번코드, 'NoneType' object has no attribute 'text'
Error in 730th obs, item: 법정동지번코드, 'NoneType' object has no attribute 'text'
Error in 730th obs, item: 지번, 'NoneType' object has no attribute 'text'
Error in 295th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 303th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 305th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 455th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 459th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'

Error in 123th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 124th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 128th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 130th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 131th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 139th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 140th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 142th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 152th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 153th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 154th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 155th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 156th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Err

...

Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 9th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 10th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 12th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 13th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명지상지하코드, 'NoneType' object has no attri

Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 22th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 27th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 30th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 32th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 34th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 35th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 37th obs, item: 도로명지상지하코드, 'NoneType' object has no att

Error in 261th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 263th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 277th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 278th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 0th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 7th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 9th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 10th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attr

Error in 1th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 23th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 24th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 26th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 30th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 32th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 33th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 35th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 39th obs, item: 도로명지상지하코드, 'NoneType' object has no attri

Error in 23th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 32th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 92th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 100th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 7th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 23th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 55th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 62th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 66th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 74th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 75th obs, item: 도로명지상지하코드, 'NoneType' object has no attr

In [9]:
# Check the data

data1 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start)+'.pkl')

data2 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 1)+'.pkl')

data3 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 2)+'.pkl')

In [10]:
data1

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"100,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),1,7,11110-2203,151.81,9,11110,9,,
1,"73,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),1,27,11110-2203,94.51,9,11110,9,,
2,"100,500",2008,2015,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),1,29,11110-2204,150.4,9-1,11110,4,,
3,"110,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),1,31,11110-2203,151.81,9,11110,10,,
4,"44,200",1981,2015,세종대로23길,00047,00000,11110,02,0,4100190,...,롯데미도파광화문빌딩,1,19,11110-12,93.36,145,11110,9,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,"7,500",2013,2015,신중남로,00001,00000,50130,02,0,3350188,...,스마트하이빌,1,14,50130-636,27.115,175-2,50130,3,,
186,"7,700",2013,2015,신중남로,00001,00000,50130,02,0,3350188,...,스마트하이빌,1,26,50130-636,27.115,175-2,50130,9,,
187,"8,000",2000,2015,비행장로55번길,00016,00000,50130,01,0,4850299,...,한빛테크,1,7,50130-35,58.944,4148-1,50130,5,,
188,"5,000",1996,2015,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,1,23,50130-44,40,1323-2,50130,3,,


In [11]:
data2

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"46,500",2007,2015,필운대로2길,00002,00000,11110,03,0,4100525,...,두레엘리시안,2,16,11110-2175,59.85,174-1,11110,7,,
1,"74,000",2007,2015,사직로,00102,00000,11110,02,0,3100005,...,신동아블루아광화문의 꿈,2,25,11110-2164,111.97,254,11110,11,,
2,"104,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),2,2,11110-2203,146.92,9,11110,12,,
3,"65,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),2,13,11110-2203,70.8,9,11110,12,,
4,"75,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),2,16,11110-2203,94.28,9,11110,11,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,"17,700",2015,2015,상모로,00315,00000,50130,00,0,3350122,...,해누리,2,9,50130-718,74,3943-4,50130,7,,
70,"16,900",2015,2015,상모로,00315,00000,50130,00,0,3350122,...,해누리,2,24,50130-718,74,3943-4,50130,3,,
71,"10,800",2000,2015,최남단해안로109번길,00070,00000,50130,01,0,4850859,...,흥도빌라트,2,13,50130-68,78.53,430-1,50130,3,,
72,"13,200",2006,2015,신영로,00016,00000,50130,01,0,3350186,...,벽성뜨래별,2,27,50130-517,81.031,1187-2,50130,5,,


In [12]:
data3

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"89,000",2000,2015,자하문로33길,00043,00000,11110,01,0,4100285,...,청운현대,3,19,11110-4,129.76,56-45,11110,1,,
1,"48,500",2002,2015,필운대로10길,00016,00000,11110,01,0,4100521,...,신현(101동),3,2,11110-7,84.82,6-13,11110,3,,
2,"85,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),3,7,11110-2203,94.51,9,11110,9,,
3,"107,000",2008,2015,경희궁길,00057,00000,11110,01,0,4100010,...,광화문풍림스페이스본(106동),3,16,11110-2204,144.52,9-1,11110,6,,
4,"73,000",2008,2015,사직로8길,00004,00000,11110,03,0,4100135,...,광화문풍림스페이스본(101동~105동),3,21,11110-2203,94.51,9,11110,7,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,"21,200",2015,2015,중문상로,00056,00000,50130,01,0,3350253,...,엘로이빌2,3,31,50130-722,78.35,2012-2,50130,3,,
65,"5,800",1996,2015,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,3,13,50130-44,40,1323-2,50130,5,,
66,"6,300",1996,2015,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,3,16,50130-44,60,1323-2,50130,5,,
67,"16,000",2010,2015,태위로603번길,00012,00000,50130,01,0,4850926,...,청우썬비치빌,3,31,50130-576,84.34,1432-4,50130,3,,
